# Neural networks code and tests notebook

# This notebook takes the flux and photon index data and try to predict classes from this data on a :

# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization,GlobalAveragePooling1D
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame


In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
#         print(lg)
        if lg==binning:
            idx_ON.append(i)

    #Future input shape : 3511 sources, 2 features (as time series) , x binnings
    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:
#3D shape of data, multivariate time series

lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []


nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))


for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']

for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][v]=dataframe['Flux']
    bigdata[1][v]=dataframe['Photon Index']



for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    
    Labels.append(int(1))


In [6]:
#Reshaping data matrix
bigdata=bigdata.reshape(c,nbfeatures,binning)

In [9]:
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(20):
    nb_filters=20
    


    weight_for_0 = (1 / lgOFF) * (lg / 2.0)
    weight_for_1 = (1 / lgON) * (lg / 2.0)
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)
    class_weight = {0: weight_for_0, 1: weight_for_1}
    bigdata=bigdata.reshape(c,binning,nbfeatures)
    #Conv1D Architecture
    model = Sequential()

    model.add(Conv1D(nb_filters, 1, padding="same", activation="tanh",input_shape=(binning,nbfeatures)))
    model.add(Conv1D(15, 1, padding="same", activation="tanh",input_shape=(nb_filters,nbfeatures)))
    model.add(Conv1D(9, 1, padding="same", activation="tanh",input_shape=(15,nbfeatures)))
    model.add(GlobalMaxPooling1D())
    
    model.add(Dense(9, activation='tanh'))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(bigdata, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)
    print('Iteration no.',i)
    #Obtain the accuradcy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.5944 - binary_accuracy: 0.6395
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5084 - binary_accuracy: 0.5626
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4788 - binary_accuracy: 0.5936
Iteration no. 0
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.5626 - binary_accuracy: 0.5884
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5201 - binary_accuracy: 0.5545
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5098 - binary_accuracy: 0.5442
Iteration no. 1
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.5688 - binary_accuracy: 0.5494
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4931 - binary_accuracy: 0.5683
Epoch 3/5
871/871 

871/871 [==============================] - 3s 3ms/step - loss: 0.5041 - binary_accuracy: 0.5752
Epoch 4/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4746 - binary_accuracy: 0.5798
Epoch 5/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4672 - binary_accuracy: 0.6234
Iteration no. 16
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 4ms/step - loss: 0.5470 - binary_accuracy: 0.6556
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4977 - binary_accuracy: 0.5580
Epoch 3/5
871/871 [==============================] - 3s 4ms/step - loss: 0.4854 - binary_accuracy: 0.5419
Iteration no. 17
7/7 [==============================] - 0s 5ms/step
Epoch 1/5
871/871 [==============================] - 5s 3ms/step - loss: 0.5601 - binary_accuracy: 0.5385
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5094 - binary_accuracy: 0.5741
Epoch 3/5
871/871 [=======